In [8]:
!pip index versions faiss-cpu

faiss-cpu (1.13.0)
Available versions: 1.13.0, 1.12.0, 1.11.0.post1, 1.11.0, 1.10.0, 1.9.0.post1, 1.9.0, 1.8.0.post1, 1.8.0


In [9]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 103.2 MB/s eta 0:00:00


In [10]:
!pip install transformers

In [11]:
!pip install pillow

In [12]:
!pip install requests

In [13]:
import json
import numpy as np
import faiss
from PIL import Image
import requests
from io import BytesIO
import torch
from transformers import CLIPProcessor, CLIPModel

In [24]:
# Load dữ liệu từ file JSON
with open('/content/sample_data/Female_updated.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

products = data['products']

In [15]:
# Load model FashionCLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("patrickjohncyh/fashion-clip").to(device)
processor = CLIPProcessor.from_pretrained("patrickjohncyh/fashion-clip")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [16]:
def get_image_embedding(image_url):
    """Lấy embedding từ ảnh"""
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content))
        inputs = processor(images=image, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        return image_features.cpu().numpy().flatten()
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh {image_url}: {str(e)}")
        return None

def truncate_text(text, max_tokens=75):
    """Cắt ngắn text để phù hợp với giới hạn token của model"""
    tokens = processor.tokenizer.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        # Decode lại tokens thành text (bỏ token đặc biệt ở đầu và cuối)
        truncated_text = processor.tokenizer.decode(tokens, skip_special_tokens=True)
        return truncated_text
    return text

def get_text_embedding(text):
    """Lấy embedding từ text"""
    try:
        # Cắt ngắn text nếu cần
        truncated_text = truncate_text(text)

        inputs = processor(text=truncated_text, return_tensors="pt", padding=True, truncation=True, max_length=77).to(device)
        with torch.no_grad():
            text_features = model.get_text_features(**inputs)
        return text_features.cpu().numpy().flatten()
    except Exception as e:
        print(f"Lỗi khi xử lý text: {str(e)}")
        print(f"Text gây lỗi (đã cắt ngắn): {truncated_text[:100]}...")
        return None

In [25]:
embeddings = []
metadata = []
failed_products = []

for i, product in enumerate(products):
    print(f"Đang xử lý sản phẩm {i+1}/{len(products)}")

    # Kết hợp các trường text một cách thông minh, ưu tiên thông tin quan trọng
    title = product.get('pdp_title_value', '')
    desc = product.get('pdp_desc_value', '')
    detail = product.get('detail_description', '')

    # Tạo text tổng hợp, ưu tiên title và mô tả ngắn
    if len(title) > 0:
        combined_text = title
        if len(desc) > 0:
            combined_text += ". " + desc
        # Chỉ thêm detail nếu còn chỗ
        if len(detail) > 0 and len(combined_text) < 200:
            combined_text += ". " + detail
    else:
        combined_text = desc if len(desc) > 0 else detail

    # Lấy embeddings
    image_embedding = get_image_embedding(product['pdp_image_url'])
    text_embedding = get_text_embedding(combined_text)

    if image_embedding is not None and text_embedding is not None:
        # Sử dụng concatenation
        combined_embedding = np.concatenate([image_embedding, text_embedding])
        embeddings.append(combined_embedding)
        metadata.append({
            'pdp_url': product['pdp_url'],
            'pdp_title': product['pdp_title_value'],
            'category': product.get('category', ''),
            'price_sp': product.get('price_sp', ''),
            'image_url': product['pdp_image_url'],
            'combined_text': combined_text[:200] + "..." if len(combined_text) > 200 else combined_text
        })
    else:
        failed_products.append({
            'index': i,
            'title': product.get('pdp_title_value', ''),
            'reason': 'image_embedding' if image_embedding is None else 'text_embedding'
        })

print(f"Xử lý thành công: {len(embeddings)}/{len(products)} sản phẩm")
print(f"Lỗi: {len(failed_products)} sản phẩm")

if len(failed_products) > 0:
    print("Các sản phẩm bị lỗi:")
    for failed in failed_products[:5]:  # Hiển thị 5 sản phẩm lỗi đầu tiên
        print(f"  - Index {failed['index']}: {failed['title']} ({failed['reason']})")

# Chuyển đổi sang numpy array
if len(embeddings) > 0:
    embeddings_array = np.array(embeddings).astype('float32')

    # Tạo FAISS index - dimension sẽ gấp đôi (512 * 2 = 1024)
    dimension = embeddings_array.shape[1]
    print(f"Dimension của embedding: {dimension}")

    index = faiss.IndexFlatIP(dimension)  # Sử dụng inner product cho similarity search

    # Chuẩn hóa vectors để inner product tương đương cosine similarity
    faiss.normalize_L2(embeddings_array)
    index.add(embeddings_array)

    # Lưu index và metadata
    faiss.write_index(index, "fashion_clip_concat_index.faiss")
    with open("metadata_concat.json", "w", encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False)

    print(f"Đã tạo index với {len(embeddings)} sản phẩm")
    print(f"Kích thước embedding: {dimension}")
    print(f"Kích thước embeddings_array: {embeddings_array.shape}")
else:
    print("Không có sản phẩm nào được xử lý thành công!")

Đang xử lý sản phẩm 1/963
Đang xử lý sản phẩm 2/963
Đang xử lý sản phẩm 3/963
Đang xử lý sản phẩm 4/963
Đang xử lý sản phẩm 5/963
Đang xử lý sản phẩm 6/963
Đang xử lý sản phẩm 7/963
Đang xử lý sản phẩm 8/963
Đang xử lý sản phẩm 9/963
Đang xử lý sản phẩm 10/963
Đang xử lý sản phẩm 11/963
Đang xử lý sản phẩm 12/963
Đang xử lý sản phẩm 13/963
Đang xử lý sản phẩm 14/963
Đang xử lý sản phẩm 15/963
Đang xử lý sản phẩm 16/963
Đang xử lý sản phẩm 17/963
Đang xử lý sản phẩm 18/963
Đang xử lý sản phẩm 19/963
Đang xử lý sản phẩm 20/963
Đang xử lý sản phẩm 21/963
Đang xử lý sản phẩm 22/963
Đang xử lý sản phẩm 23/963
Đang xử lý sản phẩm 24/963
Đang xử lý sản phẩm 25/963
Đang xử lý sản phẩm 26/963
Đang xử lý sản phẩm 27/963
Đang xử lý sản phẩm 28/963
Đang xử lý sản phẩm 29/963
Đang xử lý sản phẩm 30/963
Đang xử lý sản phẩm 31/963
Đang xử lý sản phẩm 32/963
Đang xử lý sản phẩm 33/963
Đang xử lý sản phẩm 34/963
Đang xử lý sản phẩm 35/963
Đang xử lý sản phẩm 36/963
Đang xử lý sản phẩm 37/963
Đang xử lý

In [18]:
index = faiss.read_index("fashion_clip_concat_index.faiss")
with open("metadata_concat.json", "r", encoding='utf-8') as f:
    metadata = json.load(f)

def search_similar_products(query_text, k=5):
    """Tìm kiếm sản phẩm tương tự với concatenated embeddings"""
    try:
        # Tạo text embedding cho query
        inputs = processor(text=query_text, return_tensors="pt", padding=True, truncation=True, max_length=77).to(device)
        with torch.no_grad():
            query_text_embedding = model.get_text_features(**inputs)
        query_text_embedding = query_text_embedding.cpu().numpy().astype('float32').flatten()

        # Tạo zero vector cho phần image (vì query chỉ có text)
        image_dim = 512  # FashionCLIP embedding dimension
        zero_image_embedding = np.zeros(image_dim, dtype='float32')

        # Concatenate zero image embedding với text embedding
        query_embedding = np.concatenate([zero_image_embedding, query_text_embedding])

        # Chuẩn hóa và tìm kiếm
        faiss.normalize_L2(query_embedding.reshape(1, -1))
        scores, indices = index.search(query_embedding.reshape(1, -1), k)

        # Trả về kết quả
        results = []
        for i, idx in enumerate(indices[0]):
            results.append({
                'score': scores[0][i],
                'metadata': metadata[idx]
            })
        return results
    except Exception as e:
        print(f"Lỗi khi tìm kiếm: {str(e)}")
        return []

def search_similar_products_with_image(query_image_url, k=5):
    """Tìm kiếm sản phẩm tương tự với ảnh query"""
    try:
        # Tạo image embedding cho query
        query_image_embedding = get_image_embedding(query_image_url)
        if query_image_embedding is None:
            return []

        # Tạo zero vector cho phần text (vì query chỉ có ảnh)
        text_dim = 512  # FashionCLIP embedding dimension
        zero_text_embedding = np.zeros(text_dim, dtype='float32')

        # Concatenate image embedding với zero text embedding
        query_embedding = np.concatenate([query_image_embedding, zero_text_embedding])

        # Chuẩn hóa và tìm kiếm
        faiss.normalize_L2(query_embedding.reshape(1, -1))
        scores, indices = index.search(query_embedding.reshape(1, -1), k)

        # Trả về kết quả
        results = []
        for i, idx in enumerate(indices[0]):
            results.append({
                'score': scores[0][i],
                'metadata': metadata[idx]
            })
        return results
    except Exception as e:
        print(f"Lỗi khi tìm kiếm bằng ảnh: {str(e)}")
        return []

In [19]:
def search_similar_products_with_combined(query_text=None, query_image_url=None, k=5, text_weight=0.5, image_weight=0.5):
    """
    Tìm kiếm sản phẩm tương tự với cả text và image embedding

    Args:
        query_text (str, optional): Văn bản mô tả sản phẩm cần tìm
        query_image_url (str, optional): URL ảnh sản phẩm cần tìm
        k (int): Số lượng kết quả trả về
        text_weight (float): Trọng số cho text embedding (0-1)
        image_weight (float): Trọng số cho image embedding (0-1)

    Returns:
        list: Danh sách sản phẩm tương tự với score và metadata
    """
    try:
        image_dim = 512  # FashionCLIP embedding dimension
        text_dim = 512

        # Khởi tạo embeddings
        image_embedding = np.zeros(image_dim, dtype='float32')
        text_embedding = np.zeros(text_dim, dtype='float32')

        # Tạo image embedding nếu có query_image_url
        if query_image_url is not None:
            image_embedding = get_image_embedding(query_image_url)
            if image_embedding is None:
                print("Không thể tạo image embedding, sử dụng zero vector")
                image_embedding = np.zeros(image_dim, dtype='float32')
            else:
                # Áp dụng trọng số cho image embedding
                image_embedding = image_embedding * image_weight

        # Tạo text embedding nếu có query_text
        if query_text is not None:
            inputs = processor(
                text=query_text,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=77
            ).to(device)

            with torch.no_grad():
                text_embedding = model.get_text_features(**inputs)

            text_embedding = text_embedding.cpu().numpy().astype('float32').flatten()
            # Áp dụng trọng số cho text embedding
            text_embedding = text_embedding * text_weight

        # Kiểm tra nếu không có query nào
        if query_text is None and query_image_url is None:
            print("Cần cung cấp ít nhất một trong hai: query_text hoặc query_image_url")
            return []

        # Concatenate image và text embeddings
        query_embedding = np.concatenate([image_embedding, text_embedding])

        # Chuẩn hóa embedding
        faiss.normalize_L2(query_embedding.reshape(1, -1))

        # Tìm kiếm trong FAISS index
        scores, indices = index.search(query_embedding.reshape(1, -1), k)

        # Tạo kết quả
        results = []
        for i, idx in enumerate(indices[0]):
            results.append({
                'score': float(scores[0][i]),
                'metadata': metadata[idx],
                'rank': i + 1
            })

        return results

    except Exception as e:
        print(f"Lỗi khi tìm kiếm kết hợp: {str(e)}")
        import traceback
        traceback.print_exc()
        return []

In [20]:
print("=== Tìm kiếm bằng text ===")
results = search_similar_products("áo thun nam thể thao màu nâu ngắn tay mùa đông", k=3)
for result in results:
    print(f"Score: {result['score']:.4f}")
    print(f"Title: {result['metadata']['pdp_title']}")
    print(f"Price: {result['metadata']['price_sp']}")
    print(f" URL: {result['metadata']['pdp_url']}")
    print("---")

=== Tìm kiếm bằng text ===
Score: 0.5984
Title: Áo Thun Bóng Rổ Dài Tay Nhanh Khô Cho Nam Áo Thun Thể Thao Thường Ngày Cổ Đứng Áo Thun Thể Thao Chạy Bộ Ngoài Trời Áo Thun Thể Thao
Price: 73710
 URL: https://www.lazada.vn/products/pdp-i3063700688.html
---
Score: 0.5959
Title: Quần áo đi câu bảo vệ khỏi ánh mặt trời của đàn ông ngoài trời dài tay làm khô nhanh chống tia cực tím quần áo đi câu đi câu mới
Price: 169464
 URL: https://www.lazada.vn/products/pdp-i2740862607.html
---
Score: 0.5946
Title: Quần Dài Nam Màu Trắng Mùa Hè Mỏng Thời Trang Lụa Băng Quần Dài Ống Rộng Ống Đứng Rộng Rãi Quần Dài Thường Ngày
Price: 116000
 URL: https://www.lazada.vn/products/pdp-i1974727487.html
---


In [21]:
# Thay thế bằng URL ảnh thực tế
results = search_similar_products_with_image("https://img.lazcdn.com/g/ff/kf/S26d1303cc3684401a0acf0762645efb9n.jpg_720x720q80.jpg_.webp", k=3)
for result in results:
    print(f"Score: {result['score']:.4f}")
    print(f"Title: {result['metadata']['pdp_title']}")
    print(f" URL: {result['metadata']['pdp_url']}")
    print("---")

Score: 0.7079
Title: [HCM]ÁO THUN NAM CỔ BẺ TAY NGẮN 5 MÀU ĐƠN GIÃN SANG TRỌNG DỄ MẶC VỚI NHIỀU TRANG PHỤC 01
 URL: https://www.lazada.vn/products/pdp-i254832834.html
---
Score: 0.6969
Title: [ 6 MÀU ] ÁO THUN NAM VẢI CÁ SẤU POLY DÀY MƯỢT MÁT ÁO POLO TAY NGẮN CỔ PHỐI MÀU BASIC 01
 URL: https://www.lazada.vn/products/pdp-i276994224.html
---
Score: 0.6894
Title: CÓ 3 MÀU LỰA CHỌN _ ÁO THUN NAM CÓ TÀU TAY NGẮN ĐƠN GIÃN THANH LỊCH SANG TRỌNG DỄ MẶC MÃ 12
 URL: https://www.lazada.vn/products/pdp-i2271124453.html
---


In [23]:
# 3. Tìm kiếm kết hợp cả text và image
results = search_similar_products_with_combined(
    query_text="blue t-shirt",
    query_image_url="https://img.lazcdn.com/g/ff/kf/S26d1303cc3684401a0acf0762645efb9n.jpg_720x720q80.jpg_.webp",
    k=5,
    text_weight=0.6,
    image_weight=0.4
)
for result in results:
    print(f"Score: {result['score']:.4f}")
    print(f"Title: {result['metadata']['pdp_title']}")
    print(f"Price: {result['metadata']['price_sp']}")
    print(f" URL: {result['metadata']['pdp_url']}")
    print("---")

Score: 0.7223
Title: [ 6 MÀU ] ÁO THUN NAM VẢI CÁ SẤU POLY DÀY MƯỢT MÁT ÁO POLO TAY NGẮN CỔ PHỐI MÀU BASIC 01
Price: 78000
 URL: https://www.lazada.vn/products/pdp-i276994224.html
---
Score: 0.6915
Title: Áo đũi nam cộc tay cổ bẻ full cúc
Price: 99000
 URL: https://www.lazada.vn/products/pdp-i2369924826.html
---
Score: 0.6714
Title: [HCM]ÁO THUN NAM CỔ BẺ TAY NGẮN 5 MÀU ĐƠN GIÃN SANG TRỌNG DỄ MẶC VỚI NHIỀU TRANG PHỤC 01
Price: 78000
 URL: https://www.lazada.vn/products/pdp-i254832834.html
---
Score: 0.6638
Title: áo dài tay nam thu đông đẹp có size to tới 95kg
Price: 115000
 URL: https://www.lazada.vn/products/pdp-i3177323277.html
---
Score: 0.6612
Title: CÓ 3 MÀU LỰA CHỌN _ ÁO THUN NAM CÓ TÀU TAY NGẮN ĐƠN GIÃN THANH LỊCH SANG TRỌNG DỄ MẶC MÃ 12
Price: 78000
 URL: https://www.lazada.vn/products/pdp-i2271124453.html
---
